# Confidence Intervals (CI)

A **Confidence Interval** is a range of values, derived from sample statistics, that is likely to contain the value of an unknown population parameter. While a Point Estimate gives you a single value, a Confidence Interval expresses the **uncertainty** of that estimate.

---

## 1. The Mathematical Formula
For a population mean ($\mu$), the confidence interval is calculated as:

$$CI = \bar{x} \pm \text{Margin of Error}$$
$$CI = \bar{x} \pm \left( Z^* \times \frac{\sigma}{\sqrt{n}} \right)$$

Where:
* $\bar{x}$: The Sample Mean (Point Estimate).
* $Z^*$: The **Critical Value** (determined by the confidence level).
* $\frac{\sigma}{\sqrt{n}}$: The **Standard Error (SE)**.
* $Z^* \times SE$: The **Margin of Error (MoE)**.

### Common Critical Values ($Z^*$):
* **90% Confidence:** $Z^* \approx 1.645$
* **95% Confidence:** $Z^* \approx 1.96$
* **99% Confidence:** $Z^* \approx 2.576$



---

## 2. Interpretation: What does "95% Confidence" mean?
It does **NOT** mean there is a 95% probability that the population mean lies within your specific interval.

It **DOES** mean that if you were to take 100 different samples and compute a confidence interval for each, approximately 95 of those intervals would contain the true population mean.



---

## 3. Data Science & ML Use Cases
* **Model Evaluation:** Instead of saying "Our model is 80% accurate," you say "We are 95% confident the model accuracy is between 78% and 82%."
* **A/B Testing:** Deciding if the uplift in conversion rate is large enough that the confidence intervals of the Control and Treatment groups do not overlap.
* **Risk Management:** Predicting future stock prices or energy demand within a range to account for volatility.

---

## 4. Python Implementation: Calculating CI
Use this code to calculate a 95% Confidence Interval for a set of sample data.


In [1]:

import numpy as np
from scipy import stats

# 1. Sample Data (e.g., heights of 30 people in cm)
data = [172, 168, 175, 170, 169, 174, 171, 173, 167, 170, 
        172, 169, 171, 170, 172, 168, 174, 171, 173, 170,
        169, 172, 171, 170, 175, 168, 172, 169, 171, 170]

# 2. Parameters
confidence = 0.95
n = len(data)
mean = np.mean(data)
std_err = stats.sem(data) # Standard Error of the Mean

# 3. Calculate Interval using the T-distribution (best for small/unknown samples)
h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)

start = mean - h
end = mean + h

print(f"Point Estimate (Mean): {mean:.2f}")
print(f"95% Confidence Interval: [{start:.2f}, {end:.2f}]")
print(f"Margin of Error: {h:.2f}")

Point Estimate (Mean): 170.87
95% Confidence Interval: [170.08, 171.65]
Margin of Error: 0.78


# The Z-Procedure

The **Z-procedure** is used to make inferences about a population mean ($\mu$) when the data follows a normal distribution and the population's variability is already established.

---

## 1. Mathematical Foundation
The Z-procedure relies on the **Standard Normal Distribution** ($Z \sim N(0, 1)$). 

### The Z-Interval Formula
To find the range where the true population mean likely exists:
$$\text{CI} = \bar{x} \pm Z^* \left( \frac{\sigma}{\sqrt{n}} \right)$$

### The Z-Test Statistic
To test if a sample comes from a population with a specific mean:
$$Z = \frac{\bar{x} - \mu_0}{\sigma / \sqrt{n}}$$

**Where:**
* $\bar{x}$: Sample mean.
* $\mu_0$: Hypothesized population mean.
* $\sigma$: **Known** population standard deviation.
* $n$: Sample size.
* $Z^*$: Critical value (multiplier for confidence).



---

## 2. Necessary Criteria (Assumptions)
For the Z-procedure to be valid, these three conditions must be met:
1. **Randomness:** The data must come from a random sample or randomized experiment.
2. **Normality:** Either the population is normally distributed, or the sample size is large ($n \ge 30$) so the Central Limit Theorem applies.
3. **$\sigma$ is Known:** This is the specific requirement for Z-procedures. If you only have the sample standard deviation ($s$), you must use the **T-procedure** instead.

---

## 3. How to find the Z-Critical Value ($Z^*$)
The $Z^*$ value is the number of standard deviations you must go from the mean to capture a specific percentage of the data.

| Confidence Level | Alpha ($\alpha$) | $Z^*$ Value |
| :--- | :--- | :--- |
| 90% | 0.10 | 1.645 |
| 95% | 0.05 | 1.960 |
| 99% | 0.01 | 2.576 |



---

## 4. Use Cases in Data Science & ML
* **A/B Testing (Large Scale):** When you have millions of users and a stable baseline variance, Z-tests are used to see if a new feature improved conversion rates.
* **Anomaly Detection:** Calculating how many standard deviations a new data point is from the known population mean to identify "outliers."
* **Quality Control:** In manufacturing, if the "process spread" ($\sigma$) is known from historical data, a Z-procedure checks if the current batch mean is drifting.

---

## 5. Python Implementation
This code demonstrates how to find the Z-score and the P-value for an observed sample mean.


In [3]:

import numpy as np
from scipy.stats import norm

# 1. Inputs
pop_mean = 100       # Null hypothesis mean
pop_std = 15         # Known sigma
sample_size = 50
observed_mean = 105

# 2. Calculate Z-Statistic
standard_error = pop_std / np.sqrt(sample_size)
z_score = (observed_mean - pop_mean) / standard_error

# 3. Find P-Value (Two-tailed test)
p_value = 2 * (1 - norm.cdf(abs(z_score)))

print(f"Z-Statistic: {z_score:.4f}")
print(f"P-Value: {p_value:.4f}")

if p_value < 0.05:
    print("Result is statistically significant (Reject Null)")
else:
    print("Result is not significant (Fail to Reject Null)")

Z-Statistic: 2.3570
P-Value: 0.0184
Result is statistically significant (Reject Null)


z - table - [https://www.mvcc.edu/learning-commons/pdf/table-of-areas-and-z-table.pdf]

# The T-Procedure (Student's T-test)

The **T-procedure** is a statistical method used to estimate population parameters or test hypotheses when the population standard deviation ($\sigma$) is unknown. It uses the **sample standard deviation ($s$)** as an estimate, which introduces more uncertainty, especially in smaller samples.

---

## 1. Mathematical Foundation
The T-procedure relies on the **Student's T-Distribution**. This distribution looks like a Normal curve but has **heavier tails** to account for the uncertainty of estimating the spread.

### The T-Interval Formula
Used to estimate the population mean ($\mu$):
$$\text{CI} = \bar{x} \pm t^* \left( \frac{s}{\sqrt{n}} \right)$$

### The T-Test Statistic
Used to determine if a sample mean differs significantly from a hypothesized value ($\mu_0$):
$$t = \frac{\bar{x} - \mu_0}{s / \sqrt{n}}$$

**Where:**
* $\bar{x}$: Sample mean.
* $s$: Sample standard deviation.
* $n$: Sample size.
* $t^*$: Critical value based on confidence level and **Degrees of Freedom ($df = n - 1$)**.



---

## 2. Necessary Criteria (Assumptions)
For the T-procedure to be valid, these three conditions must be met:
1. **Randomness:** Data must be collected from a random sample.
2. **Independence:** The observations must be independent (the 10% rule).
3. **Normality:** * If $n < 15$, the data must be approximately normal with no outliers.
   * If $15 \le n < 40$, the data should be symmetric with no strong outliers.
   * If $n \ge 40$, the T-procedure is robust even for skewed distributions.

---

## 3. How to Find the T-Critical Value ($t^*$)
Unlike the Z-procedure, the $t^*$ value changes depending on your sample size. As your sample size ($n$) increases, the $t^*$ value gets smaller and approaches the Z-value ($1.96$ for 95% confidence).



---

## 4. Use Cases in Data Science & ML
* **A/B Testing (Small/Medium Traffic):** Comparing the conversion rates or spend between a control and treatment group when the variance isn't known beforehand.
* **Feature Selection:** In Linear Regression, the **T-statistic** and its associated **P-value** are used to determine if a specific feature has a significant relationship with the target variable.
* **Algorithm Comparison:** If you run two different models on 10 different cross-validation folds, you use a **Paired T-test** to see if one model is truly better than the other.

---

## 5. Python Implementation: One-Sample T-test
This code calculates the T-statistic and P-value to see if a sample mean is significantly different from a target value.


In [4]:
import numpy as np
from scipy import stats

# 1. Observed Sample Data (e.g., reaction times in ms)
sample_data = [450, 470, 440, 460, 480, 455, 450, 465, 445, 475]
mu_0 = 400  # Target/Hypothesized mean

# 2. Calculate T-statistic and P-value
t_stat, p_val = stats.ttest_1samp(sample_data, mu_0)

# 3. Calculate 95% Confidence Interval
n = len(sample_data)
df = n - 1
mean = np.mean(sample_data)
sem = stats.sem(sample_data) # Standard Error of Mean (s / sqrt(n))
ci = stats.t.interval(0.95, df, loc=mean, scale=sem)

print(f"T-Statistic: {t_stat:.4f}")
print(f"P-Value: {p_val:.4e}") # Using scientific notation for small values
print(f"95% Confidence Interval: {ci}")

if p_val < 0.05:
    print("Conclusion: Reject Null Hypothesis (Difference is significant)")
else:
    print("Conclusion: Fail to Reject Null (Difference is due to luck)")

T-Statistic: 14.0370
P-Value: 2.0046e-07
95% Confidence Interval: (np.float64(449.49176117673096), np.float64(468.50823882326904))
Conclusion: Reject Null Hypothesis (Difference is significant)


t - table - [https://www.sjsu.edu/faculty/gerstman/StatPrimer/t-table.pdf]

# Deep Dive: Confidence Intervals (CI)

A **Confidence Interval** is the primary tool in inferential statistics used to estimate a population parameter with a calculated degree of uncertainty.

---

## 1. The Confidence Level ($C$)
The **Confidence Level** is the probability that the estimation method will produce an interval that contains the true population parameter if the study were repeated many times.

* **Common Levels:** 90%, 95%, and 99%.
* **Significance Level ($\alpha$):** This is the complement of the confidence level ($\alpha = 1 - C$). For a 95% confidence level, $\alpha = 0.05$. This represents the risk that the interval *does not* contain the true mean.

---

## 2. The Interval Range
The range is the actual spread between the lower and upper bounds. It is centered around your **Point Estimate** (the sample mean).

$$\text{Range} = [\text{Lower Bound}, \text{Upper Bound}]$$
$$\text{Range} = [\bar{x} - ME, \bar{x} + ME]$$

* **Point Estimate ($\bar{x}$):** The midpoint of the interval.
* **Margin of Error ($ME$):** The distance from the midpoint to either end.



---

## 3. The Margin of Error (ME)
The Margin of Error tells you the maximum expected difference between the sample statistic and the true population parameter.

**Formula for Means:**
$$ME = \text{Critical Value} \times \frac{\sigma}{\sqrt{n}}$$

### Factors Affecting the Margin of Error:
The width of your "statistical net" changes based on three variables:

| Factor | Change | Effect on $ME$ | Why? |
| :--- | :--- | :--- | :--- |
| **Confidence Level** | Increase (e.g., 95% to 99%) | **Increases** | You need a wider range to be more "certain." |
| **Sample Size ($n$)** | Increase | **Decreases** | More data provides a more precise estimate (Law of Large Numbers). |
| **Variability ($\sigma$)** | Increase | **Increases** | More "noise" in the population makes it harder to pin down the mean. |



---

## 4. Correct Interpretation
Interpretation is about the **process**, not a single specific event.

* **Correct:** "We are 95% confident that the true population mean falls within this range." (This refers to our confidence in the sampling method).
* **Incorrect:** "There is a 95% chance the mean is in this interval." (The mean isn't moving; it's either in there or it isn't).
* **Incorrect:** "95% of all individual data points fall in this range." (The interval is about the *average*, not individuals).

---

## 5. Python: Analyzing Factors of ME
Use this script to see how changing the Sample Size ($n$) directly shrinks the Margin of Error.


In [1]:
import numpy as np
from scipy import stats

def calculate_me(std_dev, n, conf_level=0.95):
    # Critical Value (Z*)
    z_critical = stats.norm.ppf((1 + conf_level) / 2)
    # Standard Error
    se = std_dev / np.sqrt(n)
    # Margin of Error
    return z_critical * se

# Scenario: Pop Std Dev is 15
sigma = 15

print(f"ME with n=30:  {calculate_me(sigma, 30):.2f}")
print(f"ME with n=100: {calculate_me(sigma, 100):.2f}")
print(f"ME with n=1000: {calculate_me(sigma, 1000):.2f}")

ME with n=30:  5.37
ME with n=100: 2.94
ME with n=1000: 0.93


link - [https://drive.google.com/file/d/1nskWHtR1ePmrje76k71gdUc2-fcVWvMH/view]